In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
# 回帰モデルを基に特徴選択
from sklearn.feature_selection import SelectFromModel

# リッジ回帰, Lasso回帰, 弾性ネット回帰
from sklearn.linear_model import Ridge, Lasso, ElasticNet
# リッジ回帰, Lasso回帰, RidgeCV, LassoCVをインポート
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV
# ElasticNet, ElasticNetCV をインポート
from sklearn.linear_model import ElasticNet, ElasticNetCV

# カテゴリ特徴量を数字に変換
def replace_to_digit(dff):
    t = list(set([v for v in dff.values])) 
    t.sort()
    vals = [t.index(v) for v in dff.values]
    return vals
# データの準備
def prepare():
    !kaggle datasets download -d \
    mylesoneill/world-university-rankings
    !unzip world-university-rankings.zip
    # 世界大学ランキングのデータを使用
    # 分類に使用する特徴量
    features = ['country', 'national_rank', 
                'quality_of_education',
                'alumni_employment', 'quality_of_faculty',
                'publications', 'influence','citations',
                'broad_impact', 'patents', 'score']
    df_train = pd.read_csv('cwurData.csv')
    df_train['country'] = \
    replace_to_digit(df_train['country'])
    X_train = df_train.loc[:,features].values
    y_train = df_train.loc[:,['world_rank']].values.ravel()
    # ビニングによりランキングを4分割
    bins = [0, 250, 500, 750, 1000]
    labels = [0, 1, 2, 3]
    classNames = ['(0,250]', '(250,500]', 
                  '(500,750]', '(750,1000]']
    y_cut = pd.cut(df_train.loc[:,['world_rank']].values.ravel(),   bins=bins, labels=labels)
    y_train = [c for c in y_cut.codes]
    return X_train, y_train, features, classNames

# 前処理（平均値による欠損値の補完、正規化）
def preprocess(X_train): 
    # 平均値による単一代入法
    simple_imp = SimpleImputer(missing_values=np.nan, 
                               strategy='mean') 
    simple_imp.fit(X_train) 
    X_train = simple_imp.transform(X_train) 
    ms = MinMaxScaler()
    ms.fit(X_train)
    X_train = ms.transform(X_train)
    return X_train

def main(): 
    X_train, y_train, features, classNames = prepare()
    print('Original Features ', features)
    X_train = preprocess(X_train)
    X_train, X_test, y_train, y_test = train_test_split( 
                                     X_train, y_train, 
                                     random_state=0, 
                                     train_size=0.9)
    # 正則化項のパラメータalphaの候補をタプルで定義
    alphas = (0.01, 0.5, 1.0) 
    cvs = [RidgeCV(alphas=alphas), 
           LassoCV(alphas=alphas), 
           ElasticNetCV(alphas=alphas)]
    selectors = [Ridge(), Lasso(), ElasticNet()]
    for cv, sel in zip(cvs, selectors): 
        # リッジ回帰、Lasso回帰、弾性ネット回帰の
        # 正則化パラメータの選択を行う
        # また、選択されたパラメータを用いた特徴選択を
        # 行い、ロジスティック回帰により学習と予測を行う
        print('Logistic Regression with {} \
        Feature Selection'.format(cv.__class__.__name__))

        cv.fit(X_train, y_train)
        print(cv.alpha_)
        sel.alpha = cv.alpha_
        s_f = SelectFromModel(sel) 
        s_f.fit(X_train, y_train)
        print('- Selected Features \
            by {} -'.format(sel.__class__.__name__))
        for i, f in enumerate(s_f.get_support()):
            if f == 1:
                print('%d: %s' % (f, features[i]))

        X_train_sel = s_f.transform(X_train)
        X_test_sel = s_f.transform(X_test)
        lr = LogisticRegression()
        lr.fit(X_train_sel, y_train)
        y_pred = lr.predict(X_test_sel)
        print('\tTest set Accuracy: %.3lf\n' % lr.score(X_test_sel, y_test) )
        print(classification_report(y_test, y_pred, target_names=classNames))


if __name__ == '__main__':
    main()


  0% 0.00/1.41M [00:00<?, ?B/s]
100% 1.41M/1.41M [00:00<00:00, 97.8MB/s]
Archive:  world-university-rankings.zip
  inflating: cwurData.csv            
  inflating: education_expenditure_supplementary_data.csv  
  inflating: educational_attainment_supplementary_data.csv  
  inflating: school_and_country_table.csv  
  inflating: shanghaiData.csv        
  inflating: timesData.csv           
Original Features  ['country', 'national_rank', 'quality_of_education', 'alumni_employment', 'quality_of_faculty', 'publications', 'influence', 'citations', 'broad_impact', 'patents', 'score']
Logistic Regression with RidgeCV         Feature Selection
0.5
- Selected Features             by Ridge -
1: alumni_employment
1: publications
1: broad_impact
1: score
	Test set Accuracy: 0.855

              precision    recall  f1-score   support

     (0,250]       0.91      0.91      0.91        68
   (250,500]       0.80      0.75      0.78        60
   (500,750]       0.74      0.80      0.77        44
  (